In [16]:
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
import pandas as pd
from selenium.webdriver.support.ui import Select
import time

In [17]:
driver = webdriver.Edge()
driver.get('https://app.cfe.mx/Aplicaciones/CCFE/Tarifas/TarifasCREIndustria/Tarifas/DemandaIndustrialSub.aspx')

KeyboardInterrupt: 

In [ ]:
year_selection = Select(driver.find_element(By.ID,'ContentPlaceHolder1_Fecha_ddAnio'))
month_selection = Select(driver.find_element(By.ID,'ContentPlaceHolder1_Fecha2_ddMes'))
state_selection = Select(driver.find_element(By.ID,'ContentPlaceHolder1_EdoMpoDiv_ddEstado'))
#municip_selection = Select(driver.find_element(By.ID,'ContentPlaceHolder1_EdoMpoDiv_ddMunicipio'))
#division_selection = Select(driver.find_element(By.ID,'ContentPlaceHolder1_EdoMpoDiv_ddDivision'))

In [ ]:
#year_options = [option.text for option in year_selection.options]
#month_options = [option.text for option in month_selection.options]
#month_options = month_options[1:]
#state_options = [option.text for option in state_selection.options]
#state_options = state_options[1:]

In [24]:
year_options = ['2022', '2021', '2020', '2019', '2018', '2017']
month_options = list(range(1,13))
state_options = list(range(1,33))

In [31]:
month_options

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]

In [40]:
driver.close()

In [41]:
driver = webdriver.Edge()
driver.get('https://app.cfe.mx/Aplicaciones/CCFE/Tarifas/TarifasCREIndustria/Tarifas/DemandaIndustrialSub.aspx')
year_selection = Select(driver.find_element(By.ID,'ContentPlaceHolder1_Fecha_ddAnio'))
year_selection.select_by_value('2022')
time.sleep(1)
for month in month_options:
    month_selection = Select(driver.find_element(By.ID,'ContentPlaceHolder1_Fecha2_ddMes'))
    month_selection.select_by_value(str(month))
    time.sleep(1)
    for state in state_options:
        state_selection = Select(driver.find_element(By.ID,'ContentPlaceHolder1_EdoMpoDiv_ddEstado'))
        state_selection.select_by_value(str(state))
        municip_selection = Select(driver.find_element(By.ID,'ContentPlaceHolder1_EdoMpoDiv_ddMunicipio'))
        municip_options = [option.text for option in municip_selection.options]
        municip_values = [option.get_attribute("value") for option in municip_selection.options]
        municip_options = municip_options[1:]
        time.sleep(1)
        for index_municip in municip_values[1:]:
            #print(index_municip)
            select_mun = Select(driver.find_element(By.ID,'ContentPlaceHolder1_EdoMpoDiv_ddMunicipio'))
            select_mun.select_by_value(str(index_municip))

            select_div = Select(driver.find_element(By.ID,'ContentPlaceHolder1_EdoMpoDiv_ddDivision'))
            div_options = [option.text for option in select_div.options]
            print(div_options)
            div_values = [option.get_attribute("value") for option in select_div.options]

            for index_division in div_values[1:]:
                select_div = Select(driver.find_element(By.ID,'ContentPlaceHolder1_EdoMpoDiv_ddDivision'))
                select_div.select_by_value(str(index_division))
                time.sleep(1)





['-- Seleccione un División --', 'BAJÍO']
['-- Seleccione un División --', 'BAJÍO']
['-- Seleccione un División --', 'BAJÍO']
['-- Seleccione un División --', 'BAJÍO']
['-- Seleccione un División --', 'BAJÍO']
['-- Seleccione un División --', 'BAJÍO']
['-- Seleccione un División --', 'BAJÍO']
['-- Seleccione un División --', 'BAJÍO']
['-- Seleccione un División --', 'BAJÍO']
['-- Seleccione un División --', 'BAJÍO']
['-- Seleccione un División --', 'BAJÍO']
['-- Seleccione un División --', 'BAJA CALIFORNIA']
['-- Seleccione un División --', 'BAJA CALIFORNIA']
['-- Seleccione un División --', 'BAJA CALIFORNIA']
['-- Seleccione un División --', 'BAJA CALIFORNIA']
['-- Seleccione un División --', 'BAJA CALIFORNIA']
['-- Seleccione un División --', 'BAJA CALIFORNIA SUR']
['-- Seleccione un División --', 'BAJA CALIFORNIA SUR']
['-- Seleccione un División --', 'BAJA CALIFORNIA SUR']
['-- Seleccione un División --', 'BAJA CALIFORNIA SUR']
['-- Seleccione un División --', 'BAJA CALIFORNIA SUR'

KeyboardInterrupt: 

In [ ]:
lista = []
for year in year_options:
    driver = webdriver.Edge()
    driver.get('https://app.cfe.mx/Aplicaciones/CCFE/Tarifas/TarifasCREIndustria/Tarifas/DemandaIndustrialSub.aspx')
    year_selection = Select(driver.find_element(By.ID,'ContentPlaceHolder1_Fecha_ddAnio'))
    year_selection.select_by_value(year)
    time.sleep(1)
    for index_month, month in enumerate(month_options):
        month_selection = Select(driver.find_element(By.ID,'ContentPlaceHolder1_Fecha2_ddMes'))
        month_selection.select_by_index(index_month)
        time.sleep(1)
        for index_state, state in enumerate(state_options):
            state_selection = Select(driver.find_element(By.ID,'ContentPlaceHolder1_EdoMpoDiv_ddEstado'))
            state_selection.select_by_index(index_state)
            municip_selection = Select(driver.find_element(By.ID,'ContentPlaceHolder1_EdoMpoDiv_ddMunicipio'))
            municip_options = [option.text for option in municip_selection.options]
            time.sleep(1)
            for index_municip, municip in enumerate(municip_options):
                municip_selection.select_by_index(index_municip)
                division_selection = Select(driver.find_element(By.ID,'ContentPlaceHolder1_EdoMpoDiv_ddDivision'))
                division_options = [option.text for option in division_selection.options]
                time.sleep(1)
                for index_division, division in enumerate(division_options):
                    division_selection.select_by_index(index_division)
                    time.sleep(1)
                    table = driver.find_element(By.CSS_SELECTOR,"table.table.table-bordered.table-striped")
                    table = table.get_attribute('outerHTML')
                    df  = pd.read_html(table)
                    df = df[0]
                    lista.append(df)

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"css selector","selector":"table.table.table-bordered.table-striped"}
  (Session info: MicrosoftEdge=107.0.1418.42)
Stacktrace:
Backtrace:
	Microsoft::Applications::Events::EventProperties::SetProperty [0x00007FF714E68502+9986]
	Microsoft::Applications::Events::EventProperty::EventProperty [0x00007FF714E05D32+1445154]
	Ordinal0 [0x00007FF7149EFC8C+654476]
	Ordinal0 [0x00007FF714A336C2+931522]
	Ordinal0 [0x00007FF714A33B10+932624]
	Ordinal0 [0x00007FF714A6FC17+1178647]
	Ordinal0 [0x00007FF714A53BDF+1063903]
	Ordinal0 [0x00007FF714A25FF4+876532]
	Ordinal0 [0x00007FF714A6CF70+1167216]
	Ordinal0 [0x00007FF714A539B3+1063347]
	Ordinal0 [0x00007FF714A2506A+872554]
	Ordinal0 [0x00007FF714A2402E+868398]
	Ordinal0 [0x00007FF714A2570F+874255]
	Microsoft::Applications::Events::EventProperty::EventProperty [0x00007FF714CC6108+135416]
	Microsoft::Applications::Events::EventProperty::EventProperty [0x00007FF714CB02C

In [ ]:
for index, month in enumerate(month_options):
    print(index)

0
1
2
3
4
5
6
7
8
9
10
11


In [ ]:
state_options

['-- Seleccione un Estado --',
 'AGUASCALIENTES',
 'BAJA CALIFORNIA',
 'BAJA CALIFORNIA SUR',
 'CAMPECHE',
 'CHIAPAS',
 'CHIHUAHUA',
 'CIUDAD DE MÉXICO',
 'COAHUILA',
 'COLIMA',
 'DURANGO',
 'ESTADO DE MÉXICO',
 'GUANAJUATO',
 'GUERRERO',
 'HIDALGO',
 'JALISCO',
 'MICHOACÁN',
 'MORELOS',
 'NAYARIT',
 'NUEVO LEÓN',
 'OAXACA',
 'PUEBLA',
 'QUERÉTARO',
 'QUINTANA ROO',
 'SAN LUIS POTOSÍ',
 'SINALOA',
 'SONORA',
 'TABASCO',
 'TAMAULIPAS',
 'TLAXCALA',
 'VERACRUZ',
 'YUCATÁN',
 'ZACATECAS']